In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

## GET INDEKS LINK

In [2]:
response = requests.get("https://www.liputan6.com/otomotif/indeks/2018/07/27?page=2")
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")

In [30]:
contentDiv = soup.findAll('div', class_="articles--list articles--list_rows")
contentDiv
a = soup.findAll('article', class_="articles--rows--item")
a['href']

TypeError: list indices must be integers or slices, not str

In [ ]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%Y/%m/%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY/mm/dd
    """
    print("page ", page)
    if cat_link == 'otomotif':
        url = "https://www.liputan6.com/"+cat_link+"/indeks/"+date+"?page="+str(page)
    else :
        url = "https://www.liputan6.com/"+cat_link+"/indeks/"+date+"?page="+str(page)
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.findAll('div', class_="latest--indeks mt2 clearfix")
    for post in contentDiv:
        link = [post.find('a', href=True)['href'], category]
        links.append(link)
        
    el_page = soup.find('div', class_="paging__wrap clearfix")
    if el_page:
        a_page = soup.find('div', class_="paging__wrap clearfix").findAll('div', class_='paging__item')[-1].find('a')
        if 'data-ci-pagination-page' in a_page:
            max_page = page
        else:
            max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, category, date)
        
    return links

## GET DETAIL BERITA

In [32]:
response = requests.get("https://www.liputan6.com/otomotif/indeks/2018/07/27?page=2")
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")

In [ ]:
articles['id'] = int(soup.find("articles--rows--item", attrs={'class':'articles--rows--item'})['content'])

In [ ]:
def getDetailBerita(url):
#     all_articles = []
#     for link in links:
    articles = {}
    #link
#         url = link[0]
    response = requests.get(url)
    html = response.text

    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")

    #extract subcategory from breadcrumb
    bc = soup.find('div', class_="breadcrumbs")
    if not bc:
        continue

    sub = bc.findAll('a')[1].text
    if ("foto" in sub.lower()) or  "video" in sub.lower(): 
        continue

    articles['subcategory'] = sub

    #articles['id'] = int(soup.find("meta", attrs={'name':'articleid'})['content'])

    #category
#         articles['category'] = link[1]
    articles['url'] = url

    article = soup.find('div', class_="tru")

    #extract date
    pubdate_author = soup.find("div",class_="reg").text
    pubdate_author_split = pubdate_author.split(' \xa0\xa0 • \xa0\xa0 ')
    pubdate = pubdate_author_split[1]
    pubdate = pubdate.strip(' \t\n\r')
    pubdate = pubdate.replace(' WIB','')
    articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%A, %d %b %Y %H:%M"), "%Y-%m-%d %H:%M:%S")

    #extract author
    author = pubdate_author_split[0]
    articles['author'] = author

    #extract title
    articles['title'] = soup.find('meta', attrs={"property":"og:title"})['content']

    #source
    articles['source'] = 'metrotvnews'

    #extract comments count
    #articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
    articles['comment'] = print("?")
    
    #extract tags
    tags = soup.find('div', class_="line").findAll('a', class_="tag")
    articles['tags'] = ','.join([x.text for x in tags])

    #extract images
    articles['images'] = soup.find('img', class_="pic")['src']

    #extract detail
    detail = soup.find('div', class_="tru")

    #hapus link sisip
    for link in detail.findAll('div', class_="related"):
        link.decompose()

    #hapus video sisip
    for tag in detail.findAll('iframe', class_="embedv"):
        tag.decompose()

    #hapus all setelah clear fix
    #for det in detail.find('div', class_="wfull fl rl"):
    #    det.decompose()

    #hapus all script
    for script in detail.findAll('script'):
        script.decompose()

    #extract content
    detail = BeautifulSoup(detail.decode_contents().replace('<br/>', ' '), "html5lib")
    content = re.sub(r'\n|\t|\b|\r','',detail.text)
    articles['content'] = content.strip(' ')
    #print('memasukkan berita id ', articles['id'])
    #all_articles.append(articles)
    return articles

In [ ]:
def getDetailBerita(links):
    all_articles = {}
    for link in links:
        #link
        articles['url'] = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")

        #articleid
        articles['id'] = int(soup.find("meta", attrs={'name':'cXenseParse:articleid'})['content'])

        #category
        articles['category'] = link[1]
        articles['url'] = url

        #extract subcategory from breadcrumb
        bc = soup.find('ul', class_="breadcrumb__wrap")
        articles['subcategory'] = bc.findAll('li')[2].text

        #article
        article = soup.find("div", class_="read__content")

        #extract date
        pubdate = soup.find("meta", attrs={"name":"content_PublishedDate"})['content']
        pubdate = pubdate.strip(' \t\n\r')
        articles['pubdate']=datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")

        #extract author
        articles['author'] = soup.find('div', class_="read__author").text

        #extract title
        articles['title'] = soup.find('h1', class_="read__title").text      

        #source
        articles['source'] = 'kompas'

        #extract comments count
#         options = Options()
#         options.add_argument('--headless')
#         options.add_argument('--disable-gpu')  # Last I checked this was necessary.
#         options.add_argument('--disable-extensions')

#         driver = webdriver.Chrome("../chromedriver.exe", chrome_options=options)
#         driver.get(url)
#         html = driver.page_source
#         soup = BeautifulSoup(html, 'html5lib')

#         comment = soup.find('div', class_="span4 comments-count tright")
#         comment_num = comment.text.replace('Ada ', '')
#         comment_num = comment_num.replace(' komentar untuk artikel ini', '')
        articles['comments'] = 0

        #extract tags
        tags = soup.find("meta", attrs={'name':'content_tag'})['content']
        articles['tags'] = tags

        #extract images
        articles['images'] = soup.find('meta', attrs={'property':'og:image'})['content']


        #hapus link sisip
        #         for link in detail.findAll('table', class_="linksisip"):
        #             link.decompose()

        #hapus video sisip
        #         for tag in detail.findAll('div', class_="detail_tag"):
        #             tag.decompose()

        #hapus all setelah clear fix
        #for det in detail.find('div', class_="wfull fl rl"):
        #     det.decompose()

        #hapus all script
        for script in article.findAll('script'):
            script.decompose()

        #extract content
        detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        #articles['content'] = re.sub('google*','', content).strip(' ')
        articles['content']
        #print('memasukkan berita id ', articles['id'])

        all_articles.append(articles)

    return all_articles